# 1.Capture datasets

### 1.1 Import dependencies

In [ ]:
import tensorflow as tf
import cv2 as cv
import uuid
import os
import time
from matplotlib import pyplot as plt
import numpy as np

### 1.2 Make directory pathes

In [ ]:
dataset_dir = 'dataset'
train_dataset_dir = 'dataset\\train'
test_dataset_dir = 'dataset\\test'
# --------------------------------- #
train_dir = 'data\\train'
test_dir = 'data\\test'
# --------------------- #
real_test_dir = 'data\\test\\real'
real_train_dir = 'data\\train\\real'
# --------------------------------- #
spoof_test_dir = 'data\\test\\spoof'
spoof_train_dir = 'data\\train\\spoof'

### 1.3 Capture images for dataset

In [ ]:
cam = cv.VideoCapture(0)
path = os.path.join('dataset')

for imgnum in range(20):
    print(f'Capturing image: {imgnum}')
    
    ret, frame = cam.read()
    img_path = os.path.join(path, f'{str(uuid.uuid1())}.jpg')
    cv.imwrite(img_path, frame)
    cv.imshow('Camera', frame)
    time.sleep(1)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv.destroyAllWindows()

### 1.4 Count images in existing folders with keras

In [ ]:
# Train folder count
train_path = os.path.join('data', 'train')
RTPath = os.path.join('data', 'train', 'real')
STPath = os.path.join('data', 'train', 'spoof')
RTLength = os.listdir(RTPath)
STLength = os.listdir(STPath)

train_data = tf.keras.utils.image_dataset_from_directory(train_path)
train_iterator = train_data.as_numpy_iterator()
train_batch = train_iterator.next()

images = len(RTLength) + len(STLength)
print(f"{images} images has been found")

In [ ]:
tr_batch = int(images / 20)
print(f"Batch size for train part: {tr_batch}")

In [ ]:
# Test folder count
test_path = os.path.join('data', 'test')
RTPath = os.path.join('data', 'test', 'real')
STPath = os.path.join('data', 'test', 'spoof')
RTLength = os.listdir(RTPath)
STLength = os.listdir(STPath)

test_data = tf.keras.utils.image_dataset_from_directory(test_path)
test_iterator = test_data.as_numpy_iterator()
test_batch = test_iterator.next()

images = len(RTLength) + len(STLength)
print(f"{images} images has been found")

In [ ]:
te_batch = int(images / 20)
print(f"Batch size for test part: {te_batch}")

### 1.5 Scale the images to make them an array with numbers between 0 and 1

In [ ]:
train_scale = train_data.map(lambda x,y: (x/255, y))
train_scaled_iterator = train_scale.as_numpy_iterator().next()[0]

maximum = train_scale.as_numpy_iterator().next()[0].max()
minimum = train_scale.as_numpy_iterator().next()[0].min()

print(f"Maximum value in train map: {maximum}")
print(f"Minimum value in train map: {minimum}")

print("-------------------------------")

test_scale = test_data.map(lambda x,y: (x/255, y))
test_scaled_iterator = test_scale.as_numpy_iterator().next()[0]

maximum = test_scale.as_numpy_iterator().next()[0].max()
minimum = test_scale.as_numpy_iterator().next()[0].min()

print(f"Maximum value in test map: {maximum}")
print(f"Minimum value in test map: {minimum}")

# 2.Data Visualization

### 2.1 Defining Functions

In [ ]:
folders = ['real', 'spoof']

In [ ]:
def get_images(file, NumberOfImages):
    img_path = []
    for types in folders:
        
        path = os.path.join(file, types)
        # E.X-1: path = os.path.join('train', 'real')
        
        counter = 1
        
        for img in os.listdir(path):
            if counter > NumberOfImages:
                break
            else:
                img_path.append(os.path.join(path, img))
                # E.X-2: img_path (which is a list) will become appended in path of E.X-1 and path of image
                
                counter = counter + 1
    
    return img_path

In [ ]:
choosen_images = get_images(file = train_dir, NumberOfImages = 25)
print(f'Number of choosen images for visualize: "{len(choosen_images)}"')
print('\n')
print(choosen_images)

In [ ]:
def visualize_dataset(img_path, rows, cols):
    fig = plt.figure(figsize = (17,17))
    
    for i in range(1, (rows*cols)+1):
        fig.add_subplot(rows, cols, i)
        img = cv.imread(img_path[i])
        
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        plt.xlabel(img_path[i].split('\\')[2])
        # E.X-3: plt.xlabel and .split command will give us a feedback about our image that it's in 'real' or 'spoof' folder 
    
    plt.show()

In [ ]:
visualize_dataset(img_path = choosen_images, rows = 4, cols = 10)

# 3.Model preparation

### 3.1 Import dependencies

In [ ]:
from keras.layers import Dense, Dropout, Input, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import model_from_json
import json

### 3.2 Different contrast and Rotation of images

In [ ]:
train_datagen = ImageDataGenerator(brightness_range=(0.8,1.2),
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest',
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   rescale=1./255)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(160,160),
                                                    color_mode='rgb',
                                                    class_mode='binary',
                                                    batch_size=tr_batch,
                                                    shuffle=True)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(160,160),
                                                  color_mode='rgb',
                                                  class_mode='binary',
                                                  batch_size=te_batch)

In [ ]:
# MobileNetV2 is a convolutional neural network architecture that seeks to perform well on mobile devices

mobilenet = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(160,160,3)))

In [ ]:
# As we are using images (not documents like PDF), we have to turn the mobilenet.trainable off
mobilenet.trainable = False

output = Flatten()(mobilenet.output)
output = Dropout(0.3)(output)
output = Dense(units = 8, activation='relu')(output)
prediction = Dense(1, activation='sigmoid')(output)

### 3.3 Check the layers of  the model with MobileNetV2

In [ ]:
model = Model(inputs = mobilenet.input, outputs = prediction)
model.summary()

### 3.4 Compile the model with Adam optimizer

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(
    learning_rate=0.000001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07),
  metrics=['accuracy'])

### 3.5 Creating a directory for "Model Checkpoint" which doesn't exist

In [ ]:
# os.mkdir('data\\model_checkpoint')

In [ ]:
model_checkpoint = ModelCheckpoint('data\\model_checkpoint\\Checkpoint model_{epoch:02d}-{val_accuracy:.6f}.h5',
                                   monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=True)


# 4.Train and Test

### 4.1 Train data to become more accurate

In [ ]:
epoch = 5

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 7,
    validation_data = test_generator, 
    validation_steps = test_generator.samples // 2,
    epochs = epoch,
    callbacks=[model_checkpoint])

### 4.2 Ploting the accuracy and loss

In [ ]:
model_json = model.to_json()
with open("finall_model_mobilenet.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
train_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

epochs = range(0, epoch)

plt.plot(epochs,train_accuracy, color='green', label='Training Accuracy')
plt.plot(epochs,validation_accuracy, color='blue', label='Validation Accuracy')

plt.title('Training & Validation Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()
plt.show()

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(0, epoch)

plt.plot(epochs, train_loss, color='red', label='Training Loss')
plt.plot(epochs, val_loss, color='orange', label='validation Loss')

plt.title('Training & Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()
plt.show()

# 5.Real Time face spoofing recognition

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import model_from_json

In [ ]:
root_dir = os.getcwd()

# Load Face Detection Model
face_cascade = cv.CascadeClassifier("models\\haarcascade_frontalface_default.xml")

# Load Anti-Spoofing Model graph
json_file = open('results.json','r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load antispoofing model weights
model.load_weights('models\\anti_spoofing_models\\antispoofing_model.h5')
print("Model loaded from directory")

video = cv.VideoCapture(0)
while video.isOpened():
    try:
        ret, frame = video.read()
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:  
            face = frame[y-5:y+h+5, x-5:x+w+5]
            resized_face = cv.resize(face,(160,160))
            resized_face = resized_face.astype("float") / 255.0
            
            # resized_face = img_to_array(resized_face)
            resized_face = np.expand_dims(resized_face, axis=0)
            # pass the face ROI through the trained liveness detector
            
            # model to determine if the face is "real" or "fake"
            preds = model.predict(resized_face)[0]
            print(preds)
            if preds > 0.5:
                label = 'spoof'
                cv.putText(frame, label, (x,y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
                cv.rectangle(frame, (x, y), (x+w,y+h),(0, 0, 255), 2)
            
            else:
                label = 'real'
                cv.putText(frame, label, (x,y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                cv.rectangle(frame, (x, y), (x+w,y+h),(0, 255, 0), 2)
        
        cv.imshow('Camera', frame)
        
        key = cv.waitKey(1)
        if key == ord('q'):
            break
    except Exception as e:
        pass

video.release()        
cv.destroyAllWindows()